# Reimplementation of original kBET for sc_04 cell type 1
- run with R(kbet)

In [1]:
#truncated normal distribution distribution function
ptnorm <- function(x, mu, sd, a=0, b=1, alpha = 0.05, verbose = FALSE){
  #this is the cumulative density of the truncated normal distribution
  #x ~ N(mu, sd^2), but we condition on a <= x <= b
  if (!is.na(x)){


    if (a > b) {
      warning("Lower and upper bound are interchanged.")
      tmp <- a
      a <- b
      b <- tmp
    }

    if (sd <= 0 || is.na(sd)) {
      if (verbose) {
        warning("Standard deviation must be positive.")
      }
      if (alpha <= 0) {
        stop("False positive rate alpha must be positive.")
      }
      sd <- alpha
    }
    if (x < a || x > b) {
      warning("x out of bounds.")
      cdf <- as.numeric(x > a)
    } else {
      alp <- pnorm((a - mu) / sd)
      bet <- pnorm((b - mu) / sd)
      zet <- pnorm((x - mu) / sd)
      cdf <- (zet - alp) / (bet - alp)
    }
    cdf
  } else {
    return(NA)
  }
}

In [2]:
# residual score function of kBET
residual_score_batch <- function(knn.set, class.freq, batch) {
    # knn.set: indices of nearest neighbors
    # class.freq: global batch proportions
    # batch: batch labels

    # return NA if all values of neighborhood are NA (which may arise from subsampling a knn-graph)
    if (all(is.na(knn.set))) {
        return(NA)
    }
    else{
        # extracts the batch labels of all neighbors (excluding NA)
        # computes local batch frequencies (observed)
        freq.env <- table(batch[knn.set[!is.na(knn.set)]]) / length(!is.na(knn.set))
        
        # create zero vector with length of batches and initialize it with freqs of batches
        full.classes <- rep(0, length(class.freq$class))
        full.classes[class.freq$class %in% names(freq.env)] <- freq.env
        
        # global batch props (expected)
        exp.freqs <- class.freq$freq

        # compute chi-square test statistics
        ## sum((observed - expected)^2 / expected)
        chi_sq_statistic <- sum((full.classes - exp.freqs)^2 / exp.freqs)
        
        return(chi_sq_statistic)
    }
}

In [3]:
# core function of kBET; called for each sample
chi_batch_test <- function(knn.set, class.freq, batch, df) {
    # knn.set: indices of nearest neighbors
    # class.freq: global batch proportions
    # batch: batch labels
    # df: degrees of freedom

    # return NA if all values of neighborhood are NA (which may arise from subsampling a knn-graph)
    if (all(is.na(knn.set))) {
        return(NA)
    }
    else {
        # extracts the batch labels of all neighbors (excluding NA)
        # computes local batch counts (observed)
        freq.env <- table(batch[knn.set[!is.na(knn.set)]])

        # create zero vector with length of batches and initialize it with counts of batches
        full.classes <- rep(0, length(class.freq$class))
        full.classes[class.freq$class %in% names(freq.env)] <- freq.env

        # global batch counts (for this sample)
        exp.freqs <- class.freq$freq * length(knn.set)

        # compute chi-square test statistics
        ## sum((observed - expected)^2 / expected)
        chi.sq.value <- sum((full.classes - exp.freqs)^2 / exp.freqs)

        # calculate p-value
        result <- 1 - pchisq(chi.sq.value, df)

        # I actually would like to know when 'NA' arises.
        if (is.na(result)) {
            return(NA)
        }
        else {
           return(result)
        }
    }
}

In [4]:
# df: dataset (rows: cells, columns: genes)
# batch: batch labels for each cell
# k0 = NULL: # neighbors to test on
# knn = NULL: n_cells x k0 matrix of kNN indices (optional)
# testSize = NULL: # data points to test (default is 10% of entire dataset, but at least 25)
# do.pca = TRUE: perform PCA before kNN search
# dim.pca = 50: if do.pca=TRUE then # dims
# heuristic = TRUE: compute an optimal neighborhood size k
# n_repeat = 100: create statistics on batch estimates running test on n_repeat subsets
# alpha = 0.05: significance level
# addTest = FALSE: perform Likelihood Ratio Test approximation to the multinimial test and
#                a multinomial exact test (if appropriate)
# verbose = FALSE: display stages of current computation
# plot = TRUE: if stats > 10 then a boxplot of the resulting rejection rates is created
#adapt = TRUE: in some cases a number of cells do not contribute to any neighborhood and 
#              this may cause an imbalance in observed and expected batch label frequencies.
#              Frequencies will be adapted if adapt=TRUE.


In [5]:
# source("kBET-utils.R")

In [6]:
library(ggplot2)

In [10]:
sc_dir <- "1/original_kbet/"
sc_dir

[1] "1/original_kbet/"

In [11]:
df <- as.matrix(read.csv(paste0(sc_dir, "df.csv"), header=FALSE))
batch <- factor(read.csv(paste0(sc_dir, "batch.csv"), header = FALSE)[[1]])
k0 <- 70
knn <- as.matrix(read.csv(paste0(sc_dir, "knn.csv"), header = FALSE))
testSize = NULL
do.pca = FALSE
dim.pca = 50
heuristic = FALSE
n_repeat = 100
alpha = 0.05
addTest = FALSE
verbose = TRUE
plot = TRUE
adapt = FALSE

dim(df)
length(batch)
k0
dim(knn)
testSize
do.pca
dim.pca
heuristic
n_repeat
alpha
addTest
verbose
plot
adapt

[1] 600  71

[1] 600

[1] 70

[1] 600  70

NULL

...

[1] 0.05

[1] FALSE

[1] TRUE

[1] TRUE

[1] FALSE

In [12]:
length(unique(batch))

[1] 2

In [13]:
is.factor(batch)

[1] TRUE

In [14]:
dof <- length(unique(batch)) - 1    # degrees of freedom

dof

[1] 1

In [15]:
if (is.factor(batch)) {
    batch <- droplevels(batch)
}

batch

[1] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [10] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [19] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [28] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [37] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [46] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [55] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [64] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [73] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [82] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [91] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[100] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[109] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[118] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[127] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[136] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[145] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[154] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[163] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[172] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[181] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[190] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[199] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[208] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[217] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[226] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[235] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[244] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[253] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[262] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[271] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[280] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[289] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[298] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[307] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[316] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[325] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[334] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[343] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[352] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[361] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[370] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[379] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[388] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[397] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[406] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[415] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[424] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[433] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[442] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[451] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[460] batch 1 batch 1 ba

In [16]:
table(batch)
length(batch)

batch
batch 1 batch 2 
    480     120 

[1] 600

In [17]:
frequencies <- table(batch) / length(batch)

frequencies

batch
batch 1 batch 2 
    0.8     0.2 

In [18]:
# get 3 different permutations of the batch label
batch.shuff <- replicate(3, batch[sample.int(length(batch))])

head(batch.shuff)

batch 1,batch 1,batch 1
batch 1,batch 1,batch 1
batch 2,batch 1,batch 1
batch 1,batch 2,batch 1
batch 2,batch 1,batch 2
batch 1,batch 1,batch 1


In [19]:
dim(batch.shuff)

[1] 600   3

In [20]:
class.frequency <- data.frame(
    class = names(frequencies),
    freq = as.numeric(frequencies)
)

class.frequency

class,freq
<chr>,<dbl>
batch 1,0.8
batch 2,0.2


In [21]:
dataset <- df

dim(dataset)

[1] 600  71

In [22]:
dim.dataset <- dim(dataset)

dim.dataset

[1] 600  71

In [23]:
dim.dataset[1]
dim.dataset[2]

[1] 600

[1] 71

In [24]:
# check dimensions
if (dim.dataset[1] != length(batch) && dim.dataset[2] != length(batch)) {
    stop("Input matrix and batch information do not match. Execution halted.")
}

In [25]:
if (dim.dataset[2] == length(batch) && dim.dataset[1] != length(batch)) {
    if (verbose) {
      cat('Input matrix has samples as columns. kBET needs samples as rows. Transposing...\n')
    }
    dataset <- t(dataset)
    dim.dataset <- dim(dataset)
  }

In [26]:
# check if the dataset is too small
if (dim.dataset[1] <= 10) {
    if (verbose) {
        cat("Your dataset has less than 10 samples. Abort and return NA.\n")
    }
    return(NA)
}

In [27]:
stopifnot(class(n_repeat) == 'numeric', n_repeat > 0)

In [28]:
# if k0 was set by the user and is too small & we do not operate on a knn graph, abort

# the reason is that if we want to test kBET on knn graph data integration methods,
# we usually face small numbers of nearest neighbours.
if (k0 < 10 & is.null(knn))
{
    if (verbose)
    {
        warning(
            "Your dataset has too few samples to run a heuristic.\n",
            "Return NA.\n",
            "Please assign k0 and set heuristic=FALSE."
        )
    }
    return(NA)
}

In [29]:
# set # tests
if (is.null(testSize) || (floor(testSize) < 1 || dim.dataset[1] < testSize))
{
    test.frac <- 0.10   # 10% of datapoints
    testSize <- ceiling(dim.dataset[1] * test.frac)
    
    if (testSize < 2 && dim.dataset[1] > 25)
    {
        testSize <- 25
    }

    if (verbose)
    {
        cat('Number of kBET tests is set to ')
        cat(paste0(testSize, '.\n'))
    }
}

Number of kBET tests is set to 60.


In [30]:
# result list
rejection <- list()
rejection$summary <- data.frame(
    kBET.expected = numeric(4),
    kBET.observed = numeric(4),
    kBET.signif = numeric(4)
)

rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0


In [31]:
rejection$results <- data.frame(
    tested = numeric(dim.dataset[1]),
    kBET.pvalue.test = rep(0, dim.dataset[1]),
    kBET.pvalue.null = rep(0, dim.dataset[1])
)

rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0


In [32]:
knn

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70
299,17,443,126,56,321,367,113,250,360,⋯,348,376,228,465,322,142,102,183,95,378
348,408,19,125,260,366,405,342,162,66,⋯,254,94,480,61,233,232,297,184,369,102
411,31,324,380,261,325,71,77,304,373,⋯,444,181,231,338,179,227,235,245,187,139
43,466,261,239,74,413,350,300,256,123,⋯,241,220,216,185,181,150,129,40,36,5
275,129,348,266,446,63,224,148,452,62,⋯,13,101,223,433,54,18,385,108,115,324
233,181,32,199,331,250,434,124,306,189,⋯,344,435,229,309,167,184,58,264,146,392
283,307,65,56,438,146,346,315,63,154,⋯,388,398,155,204,136,241,316,418,393,399
465,83,71,261,401,257,274,126,387,405,⋯,129,394,304,234,310,232,409,45,337,106
132,138,395,263,160,45,56,264,38,318,⋯,146,18,355,102,84,449,245,239,271,344
369,84,278,336,465,324,11,230,209,280,⋯,129,110,106,93,87,80,67,61,45,4


In [33]:
# concat knn matrix so
## 1st col comes 1st then 2nd col ... then k0-1 th col then cells itself
env <- as.vector(cbind(knn[, seq_len(k0 - 1)], seq_len(dim.dataset[1])))
env

[1] 299 348 411  43 275 233 283 465 132 369 325 405 357 134 208 247   1 451
   [19]  13 355 437 198  61 148 406  85 286  84 315 324 181 357 376 247 259 338
   [37] 417 398 444 367 122 172  22 316   5 142 316  36  19 371 415  48 214 101
   [55] 326 213 467 473 472 184 299  43 276 201 255 215 472 377 128 150  96 297
   [73] 311 466 389  18  61 450 291  56 286 161 405  54 342 391 159 307 379  93
   [91]  11 249  11 194 380 375 186 450 477 465 239 184 186 475 115 150 127  89
  [109] 181 326 187 465 397 416 247 398 352   4 444 261 473 346 233 309 102 405
  [127] 172 450 313 465 224 313   5 106 224 437 284 164 177 156 263 348 473  55
  [145] 466 193 338  14 371 364 270  71 257  81 116 473 305 460 405 157  36 280
  [163] 367 181  99 384  56 228 100 445 282  71 225 206   5 191 148 199 183 466
  [181] 309 397 122 369 363 405 332 124 171 466 274  18  45 181 173 406 338 204
  [199] 124 140 218 465 139 183 236 472 232 204  84 416 197 409 472 144 326 225
  [217] 193 386 445   5 235 378 403  58  93 204 435 106 106 310 129 321 427 187
  [235]   7 323 241 189 201 369 331 324 187 326 435 432 400 122 129 235 457  16
  [253]  84 435 233 466  98   3 233  19 430 318 109 463 272  25 280 229 185 280
  [271] 195  24  11 369 135  20 436 204 274 353 159  81 201 450 177 204 370  64
  [289] 398   5  27 117 472 403 294 129 127 181 448  37 337 348 369 281 106 316
  [307] 435 273 403  65 232 478 250 106 427 466 348 186 405 462 398 169 395 216
  [325] 320 369 102 443 106 282 228  18 369 409 201 454 372 369  18 301 447 473
  [343] 337 330 355 163 243 236 402 367 326 298 133 344 372 165  99 183 413  20
  [361]  24 405 122 204 157  12  18 104 156 466 320  60 201 102  29 183 125 450
  [379] 450 263 324 466 129 467 223 413 316 437 386 239 469  29  56   2 472  18
  [397] 376 181 434  41 427 422 394 220 321 376 324  52 123 233 233 419 113 342
  [415] 103 159 232 111 199 178 479 109 108 280  29  51 141  64 155 324 376  37
  [433] 263 233 228 468  11 106 248 399 181 232 338 321 310 200 339 201  38 315
  [451] 351 116 329 437 321 330  75 384 441  45 216 128 439  52 460 235 384 233
  [469] 348 356 470 203  91  46 366 368 324 131 421 301 545 537 598 531 486 540
  [487] 598 549 559 527 510 550 597 515 483 595 579 582 495 579 494 544 546 520
  [505] 530 572 541 560 492 573 486 579 519 593 598 489 597 548 490 491 551 596
  [523] 495 506 598 500 519 494 486 487 534 521 522 576 496 578 508 593 503 497
  [541] 525 593 504 589 481 576 483 503 498 490 516 529 508 507 498 504 580 541
  [559] 576 525 481 534 593 530 527 582 492 484 520 567 512 556 517 591 514 518
  [577] 580 491 538 497 565 518 517 507 588 496 514 551 580 540 593 549 547 517
  [595] 584 529 524 516 531 529  17 408  31 466 129 181 307  83 138  84 358 390
  [613] 437 276 398 360 144 163 170  50 380 132 350 261 434 242  72 371 476 112
  [631] 411 413 437 121 187 225 167  84 208   5 283  18 171 309 262 183 443 467
  [649] 405 322 167 315 398 346 473 102 324 181 323  80 401 311 115 416 337 359
  [667]  37  18  12 156 418 453 472 378 316  44 460   4 437 103 467 263  67 395
  [685] 261 311  51 261 181 357 383  12 407 240 105 466 263  18   3 437 311 130
  [703] 299 376 425 297 315 369 129 159 166   3 217 346 156 346 258  19   7 332
  [721] 319 283 405 322 280 324 346 201 200 455 451 462 114 405 409 450 480 206
  [739] 100 263 450 231 471 183 232 183  14 239 232 382 413 218 112 219 427  99
  [757] 189 416 228 299 306 119 473 206 315 263 219 342 299  36 183 450 348 389
  [775]  98 437 143 396 106 186 159 299 196  18  32   4 166 400 274 256 348 167
  [793] 122 331 183  45 233 233 252  22 127 245 437 473 319 332 419 286 478  76
  [811] 181 190  65 444 205  38 427  46 298 384 234 216 116  65  20 208 465  94
  [829] 324 351 361 306  76 360 102 277 406  24 336 263 330 326  87 375 181  20
  [847] 416 268 437 232 122 312 138 379 357  48  94 183 369  99  93 172 298 145
  [865] 427 369  69 311  65 177 462  12 398 129  38  56  20 172 455 444 437 200
  [883] 330 208 299 462   4 232 174 407 264 204  12 184 413 425 465  39 311 228
  [9

In [34]:
length(env)

[1] 42000

In [35]:
cf <- if (adapt && is.imbalanced) new.class.frequency else class.frequency
cf

class,freq
<chr>,<dbl>
batch 1,0.8
batch 2,0.2


In [36]:
length(batch)

[1] 600

In [37]:
# get avg p-value, p = 1 - CDF
rejection$average.pval <- 1 - pchisq(k0 * residual_score_batch(env, cf, batch), dof)
rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0


In [38]:
# initialize intermediates
kBET.expected <- numeric(n_repeat)
kBET.observed <- numeric(n_repeat)
kBET.signif <- numeric(n_repeat)

length(kBET.expected)
kBET.expected

[1] 100

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [39]:
testSize

[1] 60

In [40]:
alpha

[1] 0.05

In [41]:
# kBET; run n_repeat = 100 times
for (i in seq_len(n_repeat))
{
    # choose a random sample from dataset (rows: cells, cols: genes)
    # size of random sample is 10% of dataset by default
    idx.runs <- sample.int(dim.dataset[1], size=testSize)
    # print(idx.runs)
    
    # get the neighbors of sample cells; also attach sample cells as self-neighbor
    env <- cbind(knn[idx.runs, seq_len(k0 - 1)], idx.runs)
    # print(env)

    # perform test for each cell in sample
    cf <- if (adapt && is.imbalanced) new.class.frequency else class.frequency  # global batch props
    # apply chi_batch_test to each row of env matrix; 1 is rows
    p.val.test <- apply(env, 1, chi_batch_test, cf, batch, dof)
    # print(p.val.test)

    is.rejected <- p.val.test < alpha

    # apply the same test but this time batch labels are permuted. 
    # permutations are columns of batch.shuff; 2 is columns
    p.val.test.null <- apply(
        batch.shuff, 2,
        function(x) apply(env, 1, chi_batch_test, class.frequency, x, dof)
    )

    # expected rejection rate under the null for this run
    # for each shuffle compute the fraction of cells with p < alpha, then average across shuffles
    kBET.expected[i] <- mean(
        apply(
            p.val.test.null, 2,
            function(x) sum(x < alpha, na.rm = TRUE) / sum(!is.na(x))
        )
    )

    # observed rejection rate in this run
    # fraction of tested cells rejecting the null with the real labels
    kBET.observed[i] <- sum(is.rejected, na.rm = TRUE) / sum(!is.na(p.val.test))

    # compute significance
    kBET.signif[i] <- 1 - ptnorm(
        kBET.observed[i],
        mu = kBET.expected[i],
        sd = sqrt(kBET.expected[i] * (1 - kBET.expected[i]) / testSize),
        alpha = alpha
    )

    # mark which cells were sampled in this run and store
    ## their observed per-cell p-values
    ## their mean null per-cell p-values (averaged over shuffles)
    rejection$results$tested[idx.runs] <- 1
    rejection$results$kBET.pvalue.test[idx.runs] <- p.val.test
    rejection$results$kBET.pvalue.null[idx.runs] <- rowMeans(p.val.test.null, na.rm=TRUE)
}

In [42]:
rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
1,2.873077e-05,0.6934239
1,2.873077e-05,0.4900741
1,2.873077e-05,0.7717282


In [43]:
kBET.observed

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [75] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [44]:
length(kBET.observed)

[1] 100

In [45]:
sc_dir

[1] "1/original_kbet/"

In [46]:
# summarize exptected, observed and run-level significance across runs
if (n_repeat > 1)
{
    CI95 <- c(0.025, 0.5, 0.975)

    rejection$summary$kBET.expected <- c(
        mean(kBET.expected, na.rm=TRUE),
        quantile(kBET.expected, CI95, na.rm=TRUE)
    )

    rownames(rejection$summary) <- c('mean', '2.5%', '50%', '97.5%')

    rejection$summary$kBET.observed <- c(
        mean(kBET.observed, na.rm=TRUE),
        quantile(kBET.observed, CI95, na.rm=TRUE)
    )

    rejection$summary$kBET.signif <- c(
        mean(kBET.signif, na.rm=TRUE),
        quantile(kBET.signif, CI95, na.rm=TRUE)
    )

    # also return per run vectors
    rejection$stats$kBET.expected <- kBET.expected
    rejection$stats$kBET.observed <- kBET.observed
    rejection$stats$kBET.signif <- kBET.signif

    if (n_repeat < 10)
    {
        cat('Warning: The quantile computation for ')
        cat(paste0(n_repeat))
        cat(' subset results is not meaningful.')
    }

    if (plot)
    {
        plot.data <- data.frame(
            class = rep(c('observed(kBET)', 'expected(random)'), each=n_repeat),
            data = c(kBET.observed, kBET.expected)
        )

        g <- ggplot(plot.data, aes(class, data)) + geom_boxplot() + theme_bw() + labs(x = 'Test', y = 'Rejection rate') + scale_y_continuous(limits = c(0, 1))
        # print(g)
        ggsave(paste0(sc_dir, "kBET_boxplot.pdf"), plot = g, width = 7, height = 5, dpi = 300)
    }
} else
{
    rejection$summary$kBET.expected <- kBET.expected[1]
    rejection$summary$kBET.observed <- kBET.observed[1]
    rejection$summary$kBET.signif <- kBET.signif[1]
}

In [47]:
# collect parameters
rejection$params <- list()
rejection$params$k0 <- k0
rejection$params$testSize <- testSize
rejection$params$do.pca <- do.pca
rejection$params$dim.pca <- dim.pca
rejection$params$heuristic <- heuristic
rejection$params$n_repeat <- n_repeat
rejection$params$alpha <- alpha
rejection$params$addTest <- addTest
rejection$params$verbose <- verbose
rejection$params$plot <- plot

In [48]:
rejection

$summary
      kBET.expected kBET.observed kBET.signif
mean     0.01644444             1           0
2.5%     0.00000000             1           0
50%      0.01666667             1           0
97.5%    0.03333333             1           0

$results
    tested kBET.pvalue.test kBET.pvalue.null
1        1     2.873077e-05        0.6934239
2        1     2.873077e-05        0.4900741
3        1     2.873077e-05        0.7717282
4        1     2.873077e-05        0.5157274
5        1     2.873077e-05        0.8433915
6        1     2.873077e-05        0.6126951
7        1     2.873077e-05        0.3840409
8        1     2.873077e-05        0.4900741
9        1     2.873077e-05        0.7717282
10       1     2.873077e-05        0.5157274
11       1     2.873077e-05        0.5617374
12       1     2.873077e-05        0.4557213
13       1     2.873077e-05        0.6217606
14       1     2.873077e-05        0.4834503
15       1     2.873077e-05        0.4900741
16       1     2.873077e-05        0.3380309
17       1     2.873077e-05        0.6400417
18       1     2.873077e-05        0.5873907
19       1     2.873077e-05        0.4234271
20       1     2.873077e-05        0.3566943
21       1     2.873077e-05        0.2405778
22       1     2.873077e-05        0.3517638
23       1     2.873077e-05        0.7650872
24       1     2.873077e-05        0.4300509
25       1     2.873077e-05        0.7117221
26       1     2.873077e-05        0.6400417
27       1     2.873077e-05        0.3839808
28       1     2.873077e-05        0.4557042
29       1     2.873077e-05        0.3840409
30       1     2.873077e-05        0.4834503
31       1     2.873077e-05        0.6400417
32       1     2.873077e-05        0.6934239
33       1     2.873077e-05        0.5017142
34       1     2.873077e-05        0.3517638
35       1     2.873077e-05        0.4096942
36       1     2.873077e-05        0.6334007
37       1     2.873077e-05        0.5410318
38       1     2.873077e-05        0.4557042
39       1     2.873077e-05        0.2405778
40       1     2.873077e-05        0.3839979
41       1     2.873077e-05        0.7717282
42       1     2.873077e-05        0.7717282
43       1     2.873077e-05        0.7117050
44       1     2.873077e-05        0.4234271
45       1     2.873077e-05        0.4900741
46       1     2.873077e-05        0.7117050
47       1     2.873077e-05        0.6934239
48       1     2.873077e-05        0.5157274
49       1     2.873077e-05        0.7717282
50       1     2.873077e-05        0.5017313
51       1     2.873077e-05        0.3840409
52       1     2.873077e-05        0.6334178
53       1     2.873077e-05        0.5340085
54       1     2.873077e-05        0.5551136
55       1     2.873077e-05        0.5343908
56       1     2.873077e-05        0.6217606
57       1     2.873077e-05        0.4117870
58       1     2.873077e-05        0.7650872
59       1     2.873077e-05        0.5873907
60       1     2.873077e-05        0.7117050
61       1     2.873077e-05        0.5410318
62       1     2.873077e-05        0.2850310
63       1     2.873077e-05        0.1674434
64       1     2.873077e-05        0.4027043
65       1     2.873077e-05        0.6934239
66       1     2.873077e-05        0.3244172
67       1     2.873077e-05        0.5940317
68       1     2.873077e-05        0.6909993
69       1     2.873077e-05        0.5017313
70       1     2.873077e-05        0.5157274
71       1     2.873077e-05        0.6217606
72       1     2.873077e-05        0.7650872
73       1     2.873077e-05        0.6400417
74       1     2.873077e-05        0.6909993
75       1     2.873077e-05        0.7117050
76       1     2.873077e-05        0.4557042
77       1     2.873077e-05        0.6217606
78       1     2.873077e-05        0.7900092
79       1     2.873077e-05        0.3819987
80       1     2.873077e-05        0.3517638
81       1     2.873077e-05        1.0000000
82       1     2.873077e-05        0.4300509
83       1     2

In [49]:
rejection$summary$kBET.observed

[1] 1 1 1 1

In [50]:
score <- rejection$summary$kBET.observed[1]

In [51]:
score

[1] 1

In [52]:
print("End of the notebook")

[1] "End of the notebook"
